In [70]:
import typing
import random

import numpy as np

In [71]:
def sigmoid(x: np.floating):
    return 1 / (1 + np.exp(-x))

In [72]:
class Perceptron:
    def __init__(
        self,
        id: int,
        *,
        activation: typing.Callable[[np.floating], np.floating] = sigmoid
    ):
        self.id = id
        self.activation = activation

    def compute(
        self,
        inputs: np.ndarray,
        weights: np.ndarray
    ):
        bias = weights[0]
        return self.activation(
            np.dot(inputs, weights[1:]) + bias
        )

In [73]:
class InputPerceptron(Perceptron):
    def compute(self, inputs: np.ndarray, *_):
        return inputs

In [74]:
class DenseLayer:
    def __init__(self, n_perceptrons: int):
        self.perceptrons = [Perceptron(i) for i in range(n_perceptrons)]
        self.size = n_perceptrons

    def compute(
        self,
        inputs: np.ndarray,
        weights: list[np.ndarray]
    ):
        vec = []
        for idx, w in enumerate(weights):
            perceptron = self.perceptrons[idx]
            val = perceptron.compute(inputs, w)
            vec.append(val)

        return np.array(vec)


In [75]:
class InputLayer(DenseLayer):
    def __init__(self, n_perceptrons: int):
        self.perceptrons = [InputPerceptron(i) for i in range(n_perceptrons)]
        self.size = n_perceptrons

    def compute(self, inputs: np.ndarray, *_):
        return inputs

In [76]:
class Network:
    def __init__(self, layers: list[DenseLayer]):
        self.layers = layers
        self.weights: list[list[np.ndarray]] = self._initialize_random_weights()

    def _initialize_random_weights(self) -> list[list[np.ndarray]]:
        weights = []
        for idx, layer in enumerate(self.layers):
            next_layer = self.layers[idx + 1]

            intermediate_weights = []

            dim = layer.size + 1 # +1 for bias element
            num = next_layer.size

            for _ in range(num):
                vec = np.array([random.random() for _ in range(dim)])
                intermediate_weights.append(vec)

            weights.append(intermediate_weights)

            if idx + 1 == len(self.layers) - 1:
                break
        
        return weights
    
    def forward_propagate(self, X: np.ndarray):
        prop_vec = X

        for idx, layer in enumerate(self.layers):
            weights = self.weights[idx - 1]
            prop_vec = layer.compute(prop_vec, weights)

        return prop_vec



In [87]:
nn = Network([
    InputLayer(2),
    DenseLayer(1)
])

In [88]:
nn.weights

[[array([0.7627113 , 0.5958149 , 0.49293627])]]

In [89]:
X = np.array([1, 2])
nn.forward_propagate(X)

array([0.91248798])